# レビューのソート

適切な評価指標が必要

In [52]:
# 必要なライブラリをインポート
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import json

# Jupyter Notebook用の設定
%matplotlib inline

In [53]:
# データのパス
data_path = '/Users/manqueenmannequin/TW_otameshi/project_root/data/processed/android_cleaned_mecab_with_topics.csv'

# データの読み込み
df = pd.read_csv(data_path)

# データの基本情報を確認
print(f"データサイズ: {df.shape}")
df.head()

データサイズ: (1000, 21)


,id,userName,userImage,date,score,scoreText,url,title,text,replyDate,...,version,thumbsUp,criterias,positive_review,negative_review,positive_review_keywords,negative_review_keywords,topic,topic_probability,topic_name
0,9e31e6b7-605a-4b6b-9146-5a17644ba6b8,シュン,https://play-lh.googleusercontent.com/a/ACg8oc...,2024-11-23T14:01:42.496Z,1,1,https://play.google.com/store/apps/details?id=...,NaN,インストール済みアプリの広告動画で✕ボタンを押しそこねた場合、そのアプリが起動するのが凄く邪...,2024-09-16T08:11:04.465Z,...,9.6.2,70,[],-,- インストール済みアプリの広告動画で✕ボタンを押しそこねるとそのアプリが起動し、これが非常...,[],"['インストール', '済み', 'アプリ', '広告', '動画', 'ボタン', '押し...",6,0.828686,6_アプリ_こと_表示_停止
1,052823d1-61ed-4374-8a70-348fd02cd274,Asami Shikata,https://play-lh.googleusercontent.com/a/ACg8oc...,2024-11-23T11:00:06.814Z,3,3,https://play.google.com/store/apps/details?id=...,NaN,9.6.2 指摘した問題案件の改善されず。( 9.5.0 地図は表示される様になった、でも、...,2024-10-29T06:47:35.118Z,...,9.6.2,1,[{'criteria': 'vaf_never_display_disruptive_ad...,-,- 指摘した問題が改善されていない,[],"['指摘', 'し', '問題', '改善', 'さ', 'れ', 'い']",-1,0.000000,-1_いる_不満_こと_ポイント
2,2bd1806d-3415-4ac6-9db2-70800303fc48,酒井雄太,https://play-lh.googleusercontent.com/a-/ALV-U...,2024-11-22T21:54:44.660Z,4,4,https://play.google.com/store/apps/details?id=...,NaN,Wi-Fiがある場所で自動的に接続できるので便利だと思います。,NaN,...,NaN,0,[],Wi-Fiがある場所で自動的に接続できるので便利。,-,"['Wi-Fi', 'ある', '場所', '自動的', '接続', 'できる', '便利']",[],0,0.917807,0____
3,62bedb1c-1dd6-4807-8544-a04ceda38826,菅原金二郎,https://play-lh.googleusercontent.com/a/ACg8oc...,2024-11-22T20:53:26.341Z,2,2,https://play.google.com/store/apps/details?id=...,NaN,CMが多い ５秒から30秒動画が流れます,NaN,...,9.6.2,0,[],-,CMが多く、5秒から30秒の動画が流れる,[],"['CM', '多く', '5秒', '30秒', '動画', '流れる']",2,0.784345,2_動画_再生_視聴_ポイント
4,f67f4c45-f19c-4f7b-b792-6c703bb05915,みかん,https://play-lh.googleusercontent.com/a/ACg8oc...,2024-11-22T20:23:41.626Z,5,5,https://play.google.com/store/apps/details?id=...,NaN,モバイルスマホを使っていますが、ネット回線が無くてもGMOWiFiで問題なく 検索やネット回...,2024-11-25T01:08:32.912Z,...,9.6.2,0,[],ネット回線が無くてもGMOWiFiで問題なく検索やネット回線が使用できるところ。,-,"['ネット', '回線', '無く', 'GMOWiFi', '問題', '検索', 'ネッ...",[],0,0.906781,0____


In [54]:
import numpy as np
from sklearn.preprocessing import QuantileTransformer


# 日付をdatetime型に変換
# ISO8601形式に対応して日付を変換
df['date'] = pd.to_datetime(df['date'], format='ISO8601', errors='coerce')

# データ全体の最新の日付を取得
latest_date = df['date'].max()

# 各レビューに対して経過時間を計算（日数単位）
df['elapsed_days'] = (latest_date - df['date']).dt.days

# 経過日数が0の場合を防ぐために1日未満のデータを補正
df['elapsed_days'] = df['elapsed_days'].replace(0, 1)

# 共感性（Engagement）の計算
df['engagement'] = df['thumbsUp'] / df['elapsed_days']

# percentileを使って正規化

qt = QuantileTransformer(output_distribution='uniform', random_state=0)
df['engagement_quantile_normalized'] = qt.fit_transform(df[['engagement']])

# 結果を確認
df.head()

,id,userName,userImage,date,score,scoreText,url,title,text,replyDate,...,positive_review,negative_review,positive_review_keywords,negative_review_keywords,topic,topic_probability,topic_name,elapsed_days,engagement,engagement_quantile_normalized
0,9e31e6b7-605a-4b6b-9146-5a17644ba6b8,シュン,https://play-lh.googleusercontent.com/a/ACg8oc...,2024-11-23 14:01:42.496000+00:00,1,1,https://play.google.com/store/apps/details?id=...,NaN,インストール済みアプリの広告動画で✕ボタンを押しそこねた場合、そのアプリが起動するのが凄く邪...,2024-09-16T08:11:04.465Z,...,-,- インストール済みアプリの広告動画で✕ボタンを押しそこねるとそのアプリが起動し、これが非常...,[],"['インストール', '済み', 'アプリ', '広告', '動画', 'ボタン', '押し...",6,0.828686,6_アプリ_こと_表示_停止,1,70.0,0.998999
1,052823d1-61ed-4374-8a70-348fd02cd274,Asami Shikata,https://play-lh.googleusercontent.com/a/ACg8oc...,2024-11-23 11:00:06.814000+00:00,3,3,https://play.google.com/store/apps/details?id=...,NaN,9.6.2 指摘した問題案件の改善されず。( 9.5.0 地図は表示される様になった、でも、...,2024-10-29T06:47:35.118Z,...,-,- 指摘した問題が改善されていない,[],"['指摘', 'し', '問題', '改善', 'さ', 'れ', 'い']",-1,0.000000,-1_いる_不満_こと_ポイント,1,1.0,0.927928
2,2bd1806d-3415-4ac6-9db2-70800303fc48,酒井雄太,https://play-lh.googleusercontent.com/a-/ALV-U...,2024-11-22 21:54:44.660000+00:00,4,4,https://play.google.com/store/apps/details?id=...,NaN,Wi-Fiがある場所で自動的に接続できるので便利だと思います。,NaN,...,Wi-Fiがある場所で自動的に接続できるので便利。,-,"['Wi-Fi', 'ある', '場所', '自動的', '接続', 'できる', '便利']",[],0,0.917807,0____,1,0.0,0.000000
3,62bedb1c-1dd6-4807-8544-a04ceda38826,菅原金二郎,https://play-lh.googleusercontent.com/a/ACg8oc...,2024-11-22 20:53:26.341000+00:00,2,2,https://play.google.com/store/apps/details?id=...,NaN,CMが多い ５秒から30秒動画が流れます,NaN,...,-,CMが多く、5秒から30秒の動画が流れる,[],"['CM', '多く', '5秒', '30秒', '動画', '流れる']",2,0.784345,2_動画_再生_視聴_ポイント,1,0.0,0.000000
4,f67f4c45-f19c-4f7b-b792-6c703bb05915,みかん,https://play-lh.googleusercontent.com/a/ACg8oc...,2024-11-22 20:23:41.626000+00:00,5,5,https://play.google.com/store/apps/details?id=...,NaN,モバイルスマホを使っていますが、ネット回線が無くてもGMOWiFiで問題なく 検索やネット回...,2024-11-25T01:08:32.912Z,...,ネット回線が無くてもGMOWiFiで問題なく検索やネット回線が使用できるところ。,-,"['ネット', '回線', '無く', 'GMOWiFi', '問題', '検索', 'ネッ...",[],0,0.906781,0____,1,0.0,0.000000


In [55]:
# レビューの情報量を計算する
# レビューの長さを文字数で計算

# レビューの長さを計算
df['review_length'] = df['text'].str.len()

# レビューの長さをpercentileで正規化

qt = QuantileTransformer(output_distribution='uniform', random_state=0)
df['review_length_quantile_normalized'] = qt.fit_transform(df[['review_length']])
                        
# 結果を確認
df.head()

,id,userName,userImage,date,score,scoreText,url,title,text,replyDate,...,positive_review_keywords,negative_review_keywords,topic,topic_probability,topic_name,elapsed_days,engagement,engagement_quantile_normalized,review_length,review_length_quantile_normalized
0,9e31e6b7-605a-4b6b-9146-5a17644ba6b8,シュン,https://play-lh.googleusercontent.com/a/ACg8oc...,2024-11-23 14:01:42.496000+00:00,1,1,https://play.google.com/store/apps/details?id=...,NaN,インストール済みアプリの広告動画で✕ボタンを押しそこねた場合、そのアプリが起動するのが凄く邪...,2024-09-16T08:11:04.465Z,...,[],"['インストール', '済み', 'アプリ', '広告', '動画', 'ボタン', '押し...",6,0.828686,6_アプリ_こと_表示_停止,1,70.0,0.998999,323,0.971972
1,052823d1-61ed-4374-8a70-348fd02cd274,Asami Shikata,https://play-lh.googleusercontent.com/a/ACg8oc...,2024-11-23 11:00:06.814000+00:00,3,3,https://play.google.com/store/apps/details?id=...,NaN,9.6.2 指摘した問題案件の改善されず。( 9.5.0 地図は表示される様になった、でも、...,2024-10-29T06:47:35.118Z,...,[],"['指摘', 'し', '問題', '改善', 'さ', 'れ', 'い']",-1,0.000000,-1_いる_不満_こと_ポイント,1,1.0,0.927928,125,0.837838
2,2bd1806d-3415-4ac6-9db2-70800303fc48,酒井雄太,https://play-lh.googleusercontent.com/a-/ALV-U...,2024-11-22 21:54:44.660000+00:00,4,4,https://play.google.com/store/apps/details?id=...,NaN,Wi-Fiがある場所で自動的に接続できるので便利だと思います。,NaN,...,"['Wi-Fi', 'ある', '場所', '自動的', '接続', 'できる', '便利']",[],0,0.917807,0____,1,0.0,0.000000,31,0.449449
3,62bedb1c-1dd6-4807-8544-a04ceda38826,菅原金二郎,https://play-lh.googleusercontent.com/a/ACg8oc...,2024-11-22 20:53:26.341000+00:00,2,2,https://play.google.com/store/apps/details?id=...,NaN,CMが多い ５秒から30秒動画が流れます,NaN,...,[],"['CM', '多く', '5秒', '30秒', '動画', '流れる']",2,0.784345,2_動画_再生_視聴_ポイント,1,0.0,0.000000,20,0.344845
4,f67f4c45-f19c-4f7b-b792-6c703bb05915,みかん,https://play-lh.googleusercontent.com/a/ACg8oc...,2024-11-22 20:23:41.626000+00:00,5,5,https://play.google.com/store/apps/details?id=...,NaN,モバイルスマホを使っていますが、ネット回線が無くてもGMOWiFiで問題なく 検索やネット回...,2024-11-25T01:08:32.912Z,...,"['ネット', '回線', '無く', 'GMOWiFi', '問題', '検索', 'ネッ...",[],0,0.906781,0____,1,0.0,0.000000,55,0.611612


In [56]:
import MeCab
# mecabrcのパスを明示的に指定
mecab = MeCab.Tagger("-d /opt/homebrew/lib/mecab/dic/mecab-ipadic-neologd -r /opt/homebrew/etc/mecabrc")
ochasen_tagger = MeCab.Tagger("-d /opt/homebrew/lib/mecab/dic/mecab-ipadic-neologd -r /opt/homebrew/etc/mecabrc")
def lemmatize_entities(entity_list, ochasen_tagger):
    for i, entity in enumerate(entity_list):
        node = ochasen_tagger.parse(entity)
        lines = node.split("\n")
        for line in lines[:-2]:  # 最後のEOSや空行を無視
            fields = line.split("\t")
            print(fields)  # デバッグ用
            
            if len(fields) >= 2:
                # fields[1]をカンマで分割
                features = fields[1].split(",")
                # features[6]が原形（基本形）
                if len(features) > 6:
                    entity_list[i] = features[6]
    return entity_list

def extract_effective_keywords(review_document, ochasen_tagger):
    review_keywords_list = {'proper_nouns': [], 'numbers': []}
    node = ochasen_tagger.parse(review_document)
    lines = node.split("\n")

    for line in lines:
        if line == "EOS" or line.strip() == "":
            continue
        fields = line.split("\t")
        if len(fields) < 2:
            continue

        pos_info = fields[1].split(",")
        pos_major = pos_info[0]  # 名詞,動詞,形容詞など
        pos_sub1 = pos_info[1]   # 固有名詞,一般 などの細分類

        if pos_major == "名詞" and pos_sub1 == "固有名詞":
            review_keywords_list['proper_nouns'].append(fields[0])
        elif pos_major == "名詞" and pos_sub1 == "数":
            review_keywords_list['numbers'].append(fields[0])

    return review_keywords_list


In [57]:
# Example usage
review_document = '2024.09.12 アップデート後、動画が凄くスムーズに再生されるので凄く感動しています、それに貯蓄ポイントに利息が付いて来るなんて、ポイントを貯める事に熱が入るのは間違いない。今後も使いやすいアプリ目指して頑張ってください。'
keywords = extract_effective_keywords(review_document, ochasen_tagger)
print(keywords)

{'proper_nouns': ['アプリ'], 'numbers': ['2024', '09', '12']}


In [58]:
# データのpositive_review, negative_reviewに対して形態素解析を実行，結果を新たなカラムとして追加

df['negative_review_concreteness'] = df['negative_review'].apply(lambda x: extract_effective_keywords(x, ochasen_tagger))


# negative_review_concretenessに含まれる個数をカウント

df['negative_review_concreteness_count'] = df['negative_review_concreteness'].apply(lambda x: len(x['proper_nouns']) + len(x['numbers']))

# 正規化

qt = QuantileTransformer(output_distribution='uniform', random_state=0)
df['negative_review_concreteness_quantile_normalized'] = qt.fit_transform(df[['negative_review_concreteness_count']])
df.head()

,id,userName,userImage,date,score,scoreText,url,title,text,replyDate,...,topic_probability,topic_name,elapsed_days,engagement,engagement_quantile_normalized,review_length,review_length_quantile_normalized,negative_review_concreteness,negative_review_concreteness_count,negative_review_concreteness_quantile_normalized
0,9e31e6b7-605a-4b6b-9146-5a17644ba6b8,シュン,https://play-lh.googleusercontent.com/a/ACg8oc...,2024-11-23 14:01:42.496000+00:00,1,1,https://play.google.com/store/apps/details?id=...,NaN,インストール済みアプリの広告動画で✕ボタンを押しそこねた場合、そのアプリが起動するのが凄く邪...,2024-09-16T08:11:04.465Z,...,0.828686,6_アプリ_こと_表示_停止,1,70.0,0.998999,323,0.971972,"{'proper_nouns': ['アプリ', 'アプリ'], 'numbers': []}",2,0.924925
1,052823d1-61ed-4374-8a70-348fd02cd274,Asami Shikata,https://play-lh.googleusercontent.com/a/ACg8oc...,2024-11-23 11:00:06.814000+00:00,3,3,https://play.google.com/store/apps/details?id=...,NaN,9.6.2 指摘した問題案件の改善されず。( 9.5.0 地図は表示される様になった、でも、...,2024-10-29T06:47:35.118Z,...,0.000000,-1_いる_不満_こと_ポイント,1,1.0,0.927928,125,0.837838,"{'proper_nouns': [], 'numbers': []}",0,0.000000
2,2bd1806d-3415-4ac6-9db2-70800303fc48,酒井雄太,https://play-lh.googleusercontent.com/a-/ALV-U...,2024-11-22 21:54:44.660000+00:00,4,4,https://play.google.com/store/apps/details?id=...,NaN,Wi-Fiがある場所で自動的に接続できるので便利だと思います。,NaN,...,0.917807,0____,1,0.0,0.000000,31,0.449449,"{'proper_nouns': [], 'numbers': []}",0,0.000000
3,62bedb1c-1dd6-4807-8544-a04ceda38826,菅原金二郎,https://play-lh.googleusercontent.com/a/ACg8oc...,2024-11-22 20:53:26.341000+00:00,2,2,https://play.google.com/store/apps/details?id=...,NaN,CMが多い ５秒から30秒動画が流れます,NaN,...,0.784345,2_動画_再生_視聴_ポイント,1,0.0,0.000000,20,0.344845,"{'proper_nouns': ['CM', '5秒', '30秒'], 'numbers...",3,0.974975
4,f67f4c45-f19c-4f7b-b792-6c703bb05915,みかん,https://play-lh.googleusercontent.com/a/ACg8oc...,2024-11-22 20:23:41.626000+00:00,5,5,https://play.google.com/store/apps/details?id=...,NaN,モバイルスマホを使っていますが、ネット回線が無くてもGMOWiFiで問題なく 検索やネット回...,2024-11-25T01:08:32.912Z,...,0.906781,0____,1,0.0,0.000000,55,0.611612,"{'proper_nouns': [], 'numbers': []}",0,0.000000


In [59]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from bertopic import BERTopic

# モデルのロード
topic_model = BERTopic.load('/Users/manqueenmannequin/TW_otameshi/project_root/models/android_topic_model')

# 特定のキーワードリスト
keywords = ["位置情報", "許可"]

# SentenceTransformerのembedding_modelにアクセスして埋め込みを生成
keyword_embeddings = topic_model.embedding_model.embedding_model.encode(keywords)

# 各negative reviewのembeddingを取得
negative_review_embeddings = topic_model.embedding_model.embedding_model.encode(df['negative_review'])

# dfに結果を格納
df['negative_review_similarity'] = cosine_similarity(negative_review_embeddings, keyword_embeddings).max(axis=1)

In [60]:
# negative_reviewが"-"だけのものは，negative_review_similarityを0にする
df.loc[df['negative_review'] == "-", 'negative_review_similarity'] = 0

# 正規化

qt = QuantileTransformer(output_distribution='uniform', random_state=0)
df['negative_review_similarity_quantile_normalized'] = qt.fit_transform(df[['negative_review_similarity']])

# negative_review_similarity_quantile_normalizedが高い順に表示
df.sort_values('negative_review_similarity_quantile_normalized', ascending=False).head()

# 結果を確認
df.head()

,id,userName,userImage,date,score,scoreText,url,title,text,replyDate,...,elapsed_days,engagement,engagement_quantile_normalized,review_length,review_length_quantile_normalized,negative_review_concreteness,negative_review_concreteness_count,negative_review_concreteness_quantile_normalized,negative_review_similarity,negative_review_similarity_quantile_normalized
0,9e31e6b7-605a-4b6b-9146-5a17644ba6b8,シュン,https://play-lh.googleusercontent.com/a/ACg8oc...,2024-11-23 14:01:42.496000+00:00,1,1,https://play.google.com/store/apps/details?id=...,NaN,インストール済みアプリの広告動画で✕ボタンを押しそこねた場合、そのアプリが起動するのが凄く邪...,2024-09-16T08:11:04.465Z,...,1,70.0,0.998999,323,0.971972,"{'proper_nouns': ['アプリ', 'アプリ'], 'numbers': []}",2,0.924925,0.103829,0.547548
1,052823d1-61ed-4374-8a70-348fd02cd274,Asami Shikata,https://play-lh.googleusercontent.com/a/ACg8oc...,2024-11-23 11:00:06.814000+00:00,3,3,https://play.google.com/store/apps/details?id=...,NaN,9.6.2 指摘した問題案件の改善されず。( 9.5.0 地図は表示される様になった、でも、...,2024-10-29T06:47:35.118Z,...,1,1.0,0.927928,125,0.837838,"{'proper_nouns': [], 'numbers': []}",0,0.000000,0.130080,0.626627
2,2bd1806d-3415-4ac6-9db2-70800303fc48,酒井雄太,https://play-lh.googleusercontent.com/a-/ALV-U...,2024-11-22 21:54:44.660000+00:00,4,4,https://play.google.com/store/apps/details?id=...,NaN,Wi-Fiがある場所で自動的に接続できるので便利だと思います。,NaN,...,1,0.0,0.000000,31,0.449449,"{'proper_nouns': [], 'numbers': []}",0,0.000000,0.000000,0.202202
3,62bedb1c-1dd6-4807-8544-a04ceda38826,菅原金二郎,https://play-lh.googleusercontent.com/a/ACg8oc...,2024-11-22 20:53:26.341000+00:00,2,2,https://play.google.com/store/apps/details?id=...,NaN,CMが多い ５秒から30秒動画が流れます,NaN,...,1,0.0,0.000000,20,0.344845,"{'proper_nouns': ['CM', '5秒', '30秒'], 'numbers...",3,0.974975,0.120982,0.596597
4,f67f4c45-f19c-4f7b-b792-6c703bb05915,みかん,https://play-lh.googleusercontent.com/a/ACg8oc...,2024-11-22 20:23:41.626000+00:00,5,5,https://play.google.com/store/apps/details?id=...,NaN,モバイルスマホを使っていますが、ネット回線が無くてもGMOWiFiで問題なく 検索やネット回...,2024-11-25T01:08:32.912Z,...,1,0.0,0.000000,55,0.611612,"{'proper_nouns': [], 'numbers': []}",0,0.000000,0.000000,0.202202


In [61]:
# topic_probability, engagement_quantile_normalized, review_length_quantile_normalized, negative_review_concreteness_quantile_normalized, negative_review_similarity_quantile_normalizedを重み付けして，代表レビュースコアを計算

# 重み付け
topic_weight = 0.02
engagement_weight = 0.05
review_length_weight = 0.02
negative_review_concreteness_weight = 0.01
negative_review_similarity_weight = 0.9

# 重み付けしたスコアを計算
df['representative_review_score'] = (df['topic_probability'] * topic_weight) + (df['engagement_quantile_normalized'] * engagement_weight) + (df['review_length_quantile_normalized'] * review_length_weight) + (df['negative_review_concreteness_quantile_normalized'] * negative_review_concreteness_weight)

# 正規化
qt = QuantileTransformer(output_distribution='uniform', random_state=0)
df['representative_review_score_quantile_normalized'] = qt.fit_transform(df[['representative_review_score']])
df.head()


,id,userName,userImage,date,score,scoreText,url,title,text,replyDate,...,engagement_quantile_normalized,review_length,review_length_quantile_normalized,negative_review_concreteness,negative_review_concreteness_count,negative_review_concreteness_quantile_normalized,negative_review_similarity,negative_review_similarity_quantile_normalized,representative_review_score,representative_review_score_quantile_normalized
0,9e31e6b7-605a-4b6b-9146-5a17644ba6b8,シュン,https://play-lh.googleusercontent.com/a/ACg8oc...,2024-11-23 14:01:42.496000+00:00,1,1,https://play.google.com/store/apps/details?id=...,NaN,インストール済みアプリの広告動画で✕ボタンを押しそこねた場合、そのアプリが起動するのが凄く邪...,2024-09-16T08:11:04.465Z,...,0.998999,323,0.971972,"{'proper_nouns': ['アプリ', 'アプリ'], 'numbers': []}",2,0.924925,0.103829,0.547548,0.095212,0.995996
1,052823d1-61ed-4374-8a70-348fd02cd274,Asami Shikata,https://play-lh.googleusercontent.com/a/ACg8oc...,2024-11-23 11:00:06.814000+00:00,3,3,https://play.google.com/store/apps/details?id=...,NaN,9.6.2 指摘した問題案件の改善されず。( 9.5.0 地図は表示される様になった、でも、...,2024-10-29T06:47:35.118Z,...,0.927928,125,0.837838,"{'proper_nouns': [], 'numbers': []}",0,0.000000,0.130080,0.626627,0.063153,0.674675
2,2bd1806d-3415-4ac6-9db2-70800303fc48,酒井雄太,https://play-lh.googleusercontent.com/a-/ALV-U...,2024-11-22 21:54:44.660000+00:00,4,4,https://play.google.com/store/apps/details?id=...,NaN,Wi-Fiがある場所で自動的に接続できるので便利だと思います。,NaN,...,0.000000,31,0.449449,"{'proper_nouns': [], 'numbers': []}",0,0.000000,0.000000,0.202202,0.027345,0.331331
3,62bedb1c-1dd6-4807-8544-a04ceda38826,菅原金二郎,https://play-lh.googleusercontent.com/a/ACg8oc...,2024-11-22 20:53:26.341000+00:00,2,2,https://play.google.com/store/apps/details?id=...,NaN,CMが多い ５秒から30秒動画が流れます,NaN,...,0.000000,20,0.344845,"{'proper_nouns': ['CM', '5秒', '30秒'], 'numbers...",3,0.974975,0.120982,0.596597,0.032334,0.372372
4,f67f4c45-f19c-4f7b-b792-6c703bb05915,みかん,https://play-lh.googleusercontent.com/a/ACg8oc...,2024-11-22 20:23:41.626000+00:00,5,5,https://play.google.com/store/apps/details?id=...,NaN,モバイルスマホを使っていますが、ネット回線が無くてもGMOWiFiで問題なく 検索やネット回...,2024-11-25T01:08:32.912Z,...,0.000000,55,0.611612,"{'proper_nouns': [], 'numbers': []}",0,0.000000,0.000000,0.202202,0.030368,0.355355


In [62]:
similar_topics,similarity=topic_model.find_topics("許可",top_n=50)
topic_model.get_topic(similar_topics[2])

[['問題', 0.13085223925017508],
 ['許可', 0.10283327113005321],
 ['ページ', 0.10283327113005321],
 ['ある', 0.0912881483692886],
 ['状態', 0.0846821300919452],
 ['位置情報', 0.0846821300919452],
 ['続い', 0.08109358611659338],
 ['れる', 0.07727635283981181],
 ['ロード', 0.06891924107679605],
 ['頻繁', 0.06328135176746412]]

In [63]:
# 各トピックの中で，thumbsupが最も多いレビューを取得

# トピックごとにthumbsUpが最も多いレビューを取得
most_liked_reviews = df.loc[df.groupby('topic')['thumbsUp'].idxmax()]

# 結果を確認
most_liked_reviews.head()

,id,userName,userImage,date,score,scoreText,url,title,text,replyDate,...,engagement_quantile_normalized,review_length,review_length_quantile_normalized,negative_review_concreteness,negative_review_concreteness_count,negative_review_concreteness_quantile_normalized,negative_review_similarity,negative_review_similarity_quantile_normalized,representative_review_score,representative_review_score_quantile_normalized
131,76ee2b90-9101-44aa-a222-9e499e41158c,ご注文は私ですか?,https://play-lh.googleusercontent.com/a-/ALV-U...,2024-11-01 10:52:51.438000+00:00,3,3,https://play.google.com/store/apps/details?id=...,NaN,タウンワイファイを入れてから、今まで手動で切り替えていたフリーワイファイスポットも自動で接続...,2024-11-06T00:47:14.995Z,...,0.986987,341,0.973974,"{'proper_nouns': [], 'numbers': []}",0,0.000000,0.080045,0.491491,0.068829,0.767768
124,4caf10d5-6ce7-42f1-a8bb-3d203c6b5c36,hitomi,https://play-lh.googleusercontent.com/a/ACg8oc...,2024-11-02 16:12:34.246000+00:00,4,4,https://play.google.com/store/apps/details?id=...,NaN,詐欺っぽい広告ありますよね。痩せるとか目がよくなるとか。✕を何度も押させる動画は面倒だし、✕...,2024-11-06T01:06:29.261Z,...,0.997998,382,0.978979,"{'proper_nouns': [], 'numbers': []}",0,0.000000,0.000000,0.202202,0.081610,0.911912
892,124d4578-2c5d-4588-983e-f214f0210606,ござるでござる,https://play-lh.googleusercontent.com/a-/ALV-U...,2024-05-09 06:29:11.136000+00:00,5,5,https://play.google.com/store/apps/details?id=...,NaN,簡単にWi-Fiに繋いでくれるアプリでポイ活が出来るアプリがあるなんて知らなかったってのがこ...,2024-05-13T02:34:06.351Z,...,0.965966,499,1.000000,"{'proper_nouns': [], 'numbers': []}",0,0.000000,0.150575,0.676677,0.088298,0.968969
274,53d34e7c-f4e6-4286-b66c-6414d7276b9f,we are x,https://play-lh.googleusercontent.com/a/ACg8oc...,2024-10-02 20:27:05.570000+00:00,4,4,https://play.google.com/store/apps/details?id=...,NaN,全体的ににはポイントも貯まるし満足しているが。 何度も動画再生をしても、それがポイントになら...,2024-10-03T00:41:21.746Z,...,0.981982,155,0.886887,"{'proper_nouns': ['再生回数'], 'numbers': []}",1,0.795796,0.182192,0.754755,0.094795,0.993994
804,61844138-b2dd-45b8-b8da-9d8c5429a3a7,川津一臣,https://play-lh.googleusercontent.com/a/ACg8oc...,2024-05-29 01:31:25.877000+00:00,1,1,https://play.google.com/store/apps/details?id=...,NaN,・キャッチしたすべてのWi-Fiをキャッチしてポイントにしてくれるわけではない。 ・クイズの...,2023-06-05T06:12:42.241Z,...,0.961962,328,0.972973,"{'proper_nouns': ['Wi-Fi'], 'numbers': []}",1,0.795796,0.107269,0.557558,0.095516,0.996997


In [64]:
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd

# Topic 11 に該当するデータをフィルタリング
topic_id = 11
filtered_df = df[df['topic'] == topic_id].copy()

# クエリ "位置情報" をベクトル化
query = "位置情報"
query_vector = topic_model.vectorizer_model.transform([query])

# 類似度を計算して追加する関数
def calculate_similarity(review):
    # レビューが空の場合は類似度を 0 とする
    if not review or len(review.strip()) == 0:
        return 0
    
    # レビューをベクトル化
    review_vector = topic_model.vectorizer_model.transform([review])
    
    # クエリとのコサイン類似度を計算
    similarity = cosine_similarity(query_vector, review_vector)
    return similarity[0, 0]

# 各 text に対して類似度を計算
filtered_df['similarity'] = filtered_df['text'].apply(calculate_similarity)

# 類似度スコアで上位n件を取得
n = 5
top_n_reviews = filtered_df.nlargest(n, 'similarity')

# 上位5件の id と text を出力
for idx, row in top_n_reviews.iterrows():
    print(f"ID: {row['id']}")
    print(f"レビュー: {row['text']}")
    print("-" * 50)

ID: 1184b85e-9b0f-4441-a269-878107b01c1e
レビュー: 位置情報を許可してくださいの通知がしつこいけ苛々します 一時的に切っても通知来て表示を許可せんにゃ消えんし ストレス溜まりますね 自分のタイミングで許可出来る仕様にして欲しいですね… 危ないアプリじゃないですよね…？ かなり不安です
--------------------------------------------------
ID: 08b1decf-77c4-400b-9f47-f3c5bda4a5c5
レビュー: 10月から現在地入れてもｽﾀﾝﾌﾟ付かない
--------------------------------------------------
ID: 0f548332-37d4-4599-bf0d-e49c58c347ba
レビュー: 使ってて、位置確認を許可してるのに 何十メートルかずれることがあります。
--------------------------------------------------
ID: bf9d9f13-5e97-4252-8438-eaabaea31b07
レビュー: 換金出来ない
--------------------------------------------------
ID: d8062138-5859-4b1a-a7c4-476bd1be4f5b
レビュー: 今日の朝からずっと読み込みみたいなのになっていて使えません。早急になおして下さい。
--------------------------------------------------


In [67]:
# トピックごとに thumbsUp と topic probability と review length の積を計算
df['score'] = df['engagement_quantile_normalized'] + df['topic_probability'] + df['review_length_quantile_normalized']

# トピックごとに score が最も大きいレビューを取得
most_liked_reviews = df.loc[df.groupby('topic')['score'].idxmax()]

# 結果を確認

most_liked_reviews.head()

,id,userName,userImage,date,score,scoreText,url,title,text,replyDate,...,engagement_quantile_normalized,review_length,review_length_quantile_normalized,negative_review_concreteness,negative_review_concreteness_count,negative_review_concreteness_quantile_normalized,negative_review_similarity,negative_review_similarity_quantile_normalized,representative_review_score,representative_review_score_quantile_normalized
61,cbbb1b68-1c98-4540-a7b9-56b2dd8bb2c0,googleユーザー,https://play-lh.googleusercontent.com/a-/ALV-U...,2024-11-13 11:54:43.498000+00:00,1.973974,1,https://play.google.com/store/apps/details?id=...,NaN,2024.11.13 現在 もう、ポイ活を売りにするの止めれば良いのに…。【スタンプ満タン】...,2024-11-17T23:01:48.870Z,...,0.989990,423,0.983984,"{'proper_nouns': ['ボイ'], 'numbers': []}",1,0.795796,0.133275,0.637638,0.077137,0.861862
94,2aeeb515-723c-40be-98a6-3f4313fe4296,ポイ活流浪人。,https://play-lh.googleusercontent.com/a-/ALV-U...,2024-11-09 12:40:58.343000+00:00,2.936867,1,https://play.google.com/store/apps/details?id=...,NaN,アプリ起動。周辺Wi-Fiダブをタップすると多少の時間差はあるが百パーセント落ちる、その際の...,2024-11-11T05:23:25.646Z,...,0.980981,314,0.968969,"{'proper_nouns': [], 'numbers': []}",0,0.000000,0.000000,0.202202,0.088167,0.966967
892,124d4578-2c5d-4588-983e-f214f0210606,ござるでござる,https://play-lh.googleusercontent.com/a-/ALV-U...,2024-05-09 06:29:11.136000+00:00,2.965966,5,https://play.google.com/store/apps/details?id=...,NaN,簡単にWi-Fiに繋いでくれるアプリでポイ活が出来るアプリがあるなんて知らなかったってのがこ...,2024-05-13T02:34:06.351Z,...,0.965966,499,1.000000,"{'proper_nouns': [], 'numbers': []}",0,0.000000,0.150575,0.676677,0.088298,0.968969
201,04f52905-ce17-4400-b591-30de1f742ea6,Kazuhiko Okabe,https://play-lh.googleusercontent.com/a/ACg8oc...,2024-10-16 06:13:52.298000+00:00,2.970971,2,https://play.google.com/store/apps/details?id=...,NaN,ほとんどのフリーWi-Fiに自動的に繋いでくれるのでとっても便利。 ❮追記❯ ポイントが付く...,2024-10-16T08:25:00.854Z,...,0.977978,488,0.992993,"{'proper_nouns': ['大音量'], 'numbers': []}",1,0.795796,0.064170,0.465465,0.096717,1.000000
804,61844138-b2dd-45b8-b8da-9d8c5429a3a7,川津一臣,https://play-lh.googleusercontent.com/a/ACg8oc...,2024-05-29 01:31:25.877000+00:00,2.934935,1,https://play.google.com/store/apps/details?id=...,NaN,・キャッチしたすべてのWi-Fiをキャッチしてポイントにしてくれるわけではない。 ・クイズの...,2023-06-05T06:12:42.241Z,...,0.961962,328,0.972973,"{'proper_nouns': ['Wi-Fi'], 'numbers': []}",1,0.795796,0.107269,0.557558,0.095516,0.996997
